Imports

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM, Embedding
from tensorflow.keras.models import Model

from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!wget -nc https://lazyprogrammer.me/course_files/spam.csv

--2023-07-12 09:37:48--  https://lazyprogrammer.me/course_files/spam.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 172.67.213.166, 104.21.23.210, 2606:4700:3031::6815:17d2, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|172.67.213.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 503663 (492K) [text/csv]
Saving to: ‘spam.csv’

spam.csv            100%[===================>] 491.86K  1.29MB/s    in 0.4s    

2023-07-12 09:37:49 (1.29 MB/s) - ‘spam.csv’ saved [503663/503663]



In [3]:
!head spam.csv

v1,v2,,,
ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",,,
ham,Ok lar... Joking wif u oni...,,,
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,,,
ham,U dun say so early hor... U c already then say...,,,
ham,"Nah I don't think he goes to usf, he lives around here though",,,
spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, �1.50 to rcv",,,
ham,Even my brother is not like to speak with me. They treat me like aids patent.,,,
ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,,,
spam,WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim 

In [4]:
df = pd.read_csv('spam.csv', encoding='ISO-8859-1')

In [5]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Drop unnecessary columns

In [6]:
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)

In [7]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Rename columns to something better


In [8]:
df.columns = ['labels', 'data']

In [9]:
df.head()

,labels,data
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Create binary labels

In [10]:
df['b_labels'] = df.labels.map({'ham': 0, 'spam': 1})
Y = df['b_labels'].values

Split the data

In [12]:
df_train, df_test, Y_train, Y_test = train_test_split(df['data'], Y, test_size=0.33)

Convert sentences te sequences

In [18]:
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(df_train)
sequences_train = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)

get word -> integer mapping

In [19]:
word2idx = tokenizer.word_index
V = len(word2idx)
print(f"Found {V} unique tokens.")

Found 7339 unique tokens.


Pad sequences so that we get N x T matrix

In [20]:
data_train = pad_sequences(sequences_train)
print("Shape of data train tensor:", data_train.shape)

Shape of data train tensor: (3733, 189)


Get sequence length

In [21]:
T = data_train.shape[1]

In [22]:
data_test = pad_sequences(sequences_test, maxlen=T)
print("Shape of data test tensor:", data_test.shape)

Shape of data test tensor: (1839, 189)


Create the model

We get to choose embedding dimensionality (D)

Hidden state dimensionality (M)

Note: we actually want to the size of the embedding to (V + 1) x D

because the first index start from 1 and not 0

Thus, if the final index of the embedding maxtrix is V, then it actually must have size V + 1

In [23]:
D = 20
M = 15

i = Input(shape=(T, ))
x = Embedding (V + 1, D)(i)
x = LSTM(M, return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)

Compile and fit

In [24]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

r = model.fit(
    data_train, Y_train,
    validation_data=(data_test, Y_test),
    epochs=10
)

Epoch 1/10
117/117 [==============================] - 17s 116ms/step - loss: 0.4831 - accuracy: 0.8561 - val_loss: 0.3749 - val_accuracy: 0.8766
Epoch 2/10
117/117 [==============================] - 14s 122ms/step - loss: 0.4001 - accuracy: 0.8607 - val_loss: 0.3689 - val_accuracy: 0.8766
Epoch 3/10
117/117 [==============================] - 13s 107ms/step - loss: 0.3859 - accuracy: 0.8607 - val_loss: 0.3268 - val_accuracy: 0.8766
Epoch 4/10
117/117 [==============================] - 14s 120ms/step - loss: 0.2786 - accuracy: 0.8607 - val_loss: 0.2193 - val_accuracy: 0.8766
Epoch 5/10
117/117 [==============================] - 13s 107ms/step - loss: 0.1823 - accuracy: 0.9046 - val_loss: 0.1509 - val_accuracy: 0.9744
Epoch 6/10
117/117 [==============================] - 13s 109ms/step - loss: 0.1193 - accuracy: 0.9847 - val_loss: 0.1150 - val_accuracy: 0.9815
Epoch 7/10
117/117 [==============================] - 13s 110ms/step - loss: 0.0839 - accuracy: 0.9941 - val_loss: 0.0925 - val_ac